In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.layers import Flatten , Activation
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import time


#tf.config.experimental.set_visible_devices([], 'GPU')
import keras.backend.tensorflow_backend as tfback

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).
    # Source of this function: https://github.com/keras-team/keras/issues/13684
    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]



gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)
tf.config.set_visible_devices(gpus[1], 'GPU')

tfback._get_available_gpus = _get_available_gpus
print(_get_available_gpus())

                               

df =pd.read_csv('Training.csv')

df = df.replace(-999, np.nan)
df = df.dropna()
df.shape
df_droped = df.drop(['EventId', 'Weight'], axis=1)
df_droped['Label'] = df_droped['Label'].apply({'s':1, 'b':0}.get)
X_train, X_test, y_train, y_test = train_test_split(df_droped.drop(['Label'], axis=1), df_droped['Label'], test_size=0.2)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()


scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)




model = Sequential()

model.add(Flatten())


#model.add(Dense(500))
#model.add(Activation("relu")) 

#model.add(Dense(500))
#model.add(Activation("relu"))

#model.add(Dense(500))
#model.add(Activation("relu"))



#model.add(Dense(500))
#model.add(Activation('relu')) 

#model.add(Dense(500))
#model.add(Activation('relu')) 

#model.add(Dense(500))
#model.add(Activation('relu')) 

model.add(Dense(500))
model.add(Activation('relu')) 

model.add(Dense(500))
model.add(Activation('relu')) 

model.add(Dense(500))
model.add(Activation("relu"))
          
          
model.add(Dense(500))
model.add(Activation('relu'))          
          

# Add an output layer 
model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

print(X_train.shape)



#model.fit(X_train,y_train, epochs=3 ,batch_size=32)



    




['/device:GPU:0']
(54491, 30)


In [4]:
import kerastuner as kt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [5]:
import time
LOG_DIR = f"{int(time.time())}"
def build_model(hp):
    #input
    
    #model.add(Dense(hp.Int('input_units', min_value=50, max_value=5000, step=100)))
    #model.add(Activation('relu')) 
    
    for i in range(hp.Int('n_layers', 1, 10)):  # adding variation of layers.
            model.add(Dense(hp.Int(f'Dense_{i}_units',min_value=50,max_value=5000,step=100)))
            model.add(Activation('relu'))
            

    #model.add(Dense(500))
    #model.add(Activation('relu')) 

    #model.add(Dense(500))
    #model.add(Activation("relu"))
          
          
    #model.add(Dense(500))
    #model.add(Activation('relu'))          
          

    # Add an output layer 
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model


In [6]:
LOG_DIR = f"{int(time.time())}"

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,  # how many model variations to test?
    executions_per_trial=3,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

tuner.search(x=X_train,
             y=y_train,
             verbose=2, 
             epochs=10,
             batch_size=64,
            
             validation_data=(X_test, y_test))

tuner.get_best_hyperparameters()[0].values
tuner.get_best_models()[0].summary()
tuner.results_summary()

Train on 54491 samples, validate on 13623 samples
Epoch 1/10
54491/54491 - 7s - loss: 0.4362 - accuracy: 0.7886 - val_loss: 0.3809 - val_accuracy: 0.8322
Epoch 2/10
54491/54491 - 7s - loss: 0.3663 - accuracy: 0.8375 - val_loss: 0.3604 - val_accuracy: 0.8424
Epoch 3/10
54491/54491 - 7s - loss: 0.3557 - accuracy: 0.8432 - val_loss: 0.3569 - val_accuracy: 0.8453
Epoch 4/10
54491/54491 - 6s - loss: 0.3503 - accuracy: 0.8456 - val_loss: 0.3573 - val_accuracy: 0.8434
Epoch 5/10
54491/54491 - 6s - loss: 0.3450 - accuracy: 0.8482 - val_loss: 0.3551 - val_accuracy: 0.8439
Epoch 6/10
54491/54491 - 6s - loss: 0.3395 - accuracy: 0.8507 - val_loss: 0.3564 - val_accuracy: 0.8418
Epoch 7/10
54491/54491 - 6s - loss: 0.3344 - accuracy: 0.8527 - val_loss: 0.3548 - val_accuracy: 0.8453
Epoch 8/10
54491/54491 - 6s - loss: 0.3301 - accuracy: 0.8543 - val_loss: 0.3578 - val_accuracy: 0.8392
Epoch 9/10
54491/54491 - 6s - loss: 0.3248 - accuracy: 0.8577 - val_loss: 0.3603 - val_accuracy: 0.8445
Epoch 10/10
54

INFO:tensorflow:Oracle triggered exit
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_5 (Dense)              multiple                  15500     
_________________________________________________________________
activation_5 (Activation)    multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  250500    
_________________________________________________________________
activation_6 (Activation)    multiple                  0         
_________________________________________________________________
dense_7 (Dense)              multiple                  250500    
_________________________________________________________________
activation_7 (Ac

In [8]:
import pickle
#tuner.get_best_hyperparameters()[0].values
#tuner.get_best_models()[0].summary()
tuner.results_summary()